In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import tensorflow as tf
import keras

import os

In [2]:
kSZFiles = os.listdir('./Data/kSZPS')
velFiles = os.listdir('./Data/velocityFieldPS')
densFiles = os.listdir('./Data/overdensityPS')

nside = 64
lmax = 50
numSets=len(densFiles)
almSize = hp.sphtfunc.Alm.getsize(lmax)

In [3]:
overdensData = np.zeros((numSets,almSize,2))
kSZData = np.zeros((numSets,almSize,2))
velData = np.zeros((numSets,almSize,2))

In [4]:
for i in range(0,numSets):
    overdensData[i] = np.load('./Data/overdensityPS/'+densFiles[i])
    kSZData[i] = np.load('./Data/kSZPS/'+kSZFiles[i])
    velData[i] = np.load('./Data/velocityFieldPS/'+velFiles[i])

In [5]:
x_raw = np.stack((overdensData,kSZData),axis=2)
x_raw = np.reshape(x_raw,(numSets,almSize,4))

x_train, x_test = np.split(x_raw, indices_or_sections=[numSets-numSets//20])
y_train, y_test = np.split(velData, indices_or_sections=[numSets-numSets//20])

In [6]:
dset_test = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(x_test),
                                 tf.data.Dataset.from_tensor_slices(y_test)))
dset_test = dset_test.shuffle(20)
dset_test = dset_test.batch(8)

dset_train = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(x_train),
                                  tf.data.Dataset.from_tensor_slices(y_train)))
dset_train = dset_train.shuffle(200)
dset_train = dset_train.batch(8)

2023-11-15 16:05:32.800745: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

dset_train = dset_train.prefetch(buffer_size=AUTOTUNE)
dset_test = dset_test.prefetch(buffer_size=AUTOTUNE)

In [31]:
model = keras.Sequential(
    [   keras.layers.Flatten(),
        keras.layers.Dense(2*almSize, activation="linear", name="layer1"),
        keras.layers.Dense(2*almSize, activation="linear", name="layer2"),
        #keras.layers.Dense(2*almSize, activation="linear",name="layer3"),
        keras.layers.Reshape((almSize,2))
    ]
)

In [32]:
model.build(input_shape=(None, almSize,4))
model.summary(110)

Model: "sequential_4"
______________________________________________________________________________________________________________
 Layer (type)                                    Output Shape                                Param #          
 flatten_4 (Flatten)                             (None, 5304)                                0                
                                                                                                              
 layer1 (Dense)                                  (None, 2652)                                14068860         
                                                                                                              
 layer2 (Dense)                                  (None, 2652)                                7035756          
                                                                                                              
 reshape_4 (Reshape)                             (None, 1326, 2)                          

In [33]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [34]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

model.compile(optimizer=opt,
              #loss=tf.keras.losses.Huber(delta=1.0,reduction="auto",name="huber_loss"),
                loss="MSE",
              metrics=[r_square]
)


In [35]:
initial_epochs = 20
history = model.fit(
    dset_train,
    epochs=initial_epochs,
    validation_data = dset_test,
)

Epoch 1/20
177/177 [==============================] - 4s 22ms/step - loss: 13894.0381 - r_square: -2.9159e-05 - val_loss: 14218.1670 - val_r_square: -8.1640e-05
Epoch 2/20
177/177 [==============================] - 4s 21ms/step - loss: 13886.0723 - r_square: 5.3697e-04 - val_loss: 14223.4033 - val_r_square: -2.5361e-04
Epoch 3/20
177/177 [==============================] - 4s 21ms/step - loss: 13847.9395 - r_square: 0.0033 - val_loss: 14236.0977 - val_r_square: -6.1032e-04
Epoch 4/20
177/177 [==============================] - 4s 22ms/step - loss: 13765.8203 - r_square: 0.0090 - val_loss: 14257.8838 - val_r_square: -0.0030
Epoch 5/20
177/177 [==============================] - 4s 21ms/step - loss: 13644.1846 - r_square: 0.0175 - val_loss: 14282.3613 - val_r_square: -0.0038
Epoch 6/20
177/177 [==============================] - 4s 20ms/step - loss: 13489.0234 - r_square: 0.0285 - val_loss: 14306.6484 - val_r_square: -0.0084
Epoch 7/20
177/177 [==============================] - 4s 21ms/step 

In [36]:
fine_tune_epochs = 80
total_epochs =  initial_epochs + fine_tune_epochs

history_further = model.fit(dset_train,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=dset_test)

Epoch 20/100
177/177 [==============================] - 4s 21ms/step - loss: 9842.0615 - r_square: 0.2817 - val_loss: 15381.2734 - val_r_square: -0.0658
Epoch 21/100
177/177 [==============================] - 4s 21ms/step - loss: 9623.6631 - r_square: 0.2943 - val_loss: 15492.0625 - val_r_square: -0.0970
Epoch 22/100
177/177 [==============================] - 3s 19ms/step - loss: 9417.0742 - r_square: 0.3086 - val_loss: 15609.4424 - val_r_square: -0.0964
Epoch 23/100
177/177 [==============================] - 4s 20ms/step - loss: 9219.1484 - r_square: 0.3248 - val_loss: 15722.6064 - val_r_square: -0.0960
Epoch 24/100
177/177 [==============================] - 3s 19ms/step - loss: 9027.0273 - r_square: 0.3382 - val_loss: 15858.3359 - val_r_square: -0.1563
Epoch 25/100
177/177 [==============================] - 4s 20ms/step - loss: 8843.1689 - r_square: 0.3489 - val_loss: 15971.3057 - val_r_square: -0.1266
Epoch 26/100
177/177 [==============================] - 4s 20ms/step - loss: 8668.

KeyboardInterrupt: 

In [19]:
total_epochs =  initial_epochs + fine_tune_epochs+80

history_furthest = model.fit(dset_train,
                         epochs=total_epochs,
                         initial_epoch=history_further.epoch[-1],
                         validation_data=dset_test)

Epoch 100/180
177/177 [==============================] - 25s 139ms/step - loss: 268.0441 - val_loss: 19984.6465
Epoch 101/180
177/177 [==============================] - 25s 140ms/step - loss: 265.5523 - val_loss: 19989.2715
Epoch 102/180
177/177 [==============================] - 25s 141ms/step - loss: 258.0743 - val_loss: 20089.6035
Epoch 103/180
177/177 [==============================] - 25s 144ms/step - loss: 253.3967 - val_loss: 19990.1250
Epoch 104/180
177/177 [==============================] - 25s 140ms/step - loss: 244.8819 - val_loss: 19956.8926
Epoch 105/180
177/177 [==============================] - 25s 141ms/step - loss: 237.1285 - val_loss: 20039.2344
Epoch 106/180
177/177 [==============================] - 25s 143ms/step - loss: 228.6740 - val_loss: 19982.4180
Epoch 107/180
177/177 [==============================] - 24s 135ms/step - loss: 218.7421 - val_loss: 20251.9199
Epoch 108/180
177/177 [==============================] - 25s 140ms/step - loss: 211.8487 - val_loss: 199

KeyboardInterrupt: 